In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langgraph.graph import StateGraph, START
from typing_extensions import TypedDict, List
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from supabase.client import create_client

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Step 1: Create a function for Deepseek Call

In [2]:
import requests
import os
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_URL = "https://api.deepseek.com/v1/chat/completions"

def call_deepseek(question, context):
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{question}"}
    ]
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 512
    }

    response = requests.post(DEEPSEEK_URL, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]


In [5]:
response=call_deepseek("best food to have","I am in Dallas, Texas.")
print(response)

Dallas, Texas, is known for its vibrant food scene, blending Southern comfort food, Tex-Mex, barbecue, and modern culinary innovations. Here are some must-try foods in Dallas:

### 1. **Texas BBQ**  
   - **What to try**: Brisket (especially the moist cut), ribs, sausage, and smoked turkey.  
   - **Where to go**:  
     - **Pecan Lodge** (Deep Ellum) – Famous for its brisket and "Hot Mess" (sweet potato with brisket).  
     - **Terry Black’s BBQ** (Deep Ellum) – Renowned for melt-in-your-mouth beef ribs.  
     - **Lockhart Smokehouse** (Bishop Arts) – Authentic Central Texas-style BBQ.

### 2. **Tex-Mex**  
   - **What to try**: Breakfast tacos, queso, enchiladas, and fajitas.  
   - **Where to go**:  
     - **Mi Cocina** (Highland Park) – Known for its "Mambo Taxi" margarita and Tex-Mex classics.  
     - **El Fenix** (Downtown) – A Dallas staple since 1918.  
     - **Torchy’s Tacos** (Multiple locations) – Famous for creative tacos like the "Trailer Park" (fried chicken).  

###

# Step 2: Load documents, Split, and Index.

In [3]:
#url="https://www.healthline.com/health/bodyweight-workout"
url="http://www.plansum.cn/gerenjihua/73374.html"
loader = WebBaseLoader([url])
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=15)
splits = splitter.split_documents(docs)
 
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipykernel_24352/2293971527.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
splits

[Document(metadata={'source': 'http://www.plansum.cn/gerenjihua/73374.html', 'title': '健身房健身计划(精选5篇)', 'description': '胸部、肱三头肌、腹部胸部：杠铃卧推3/组12/次哑铃飞鸟3/组12/次俯卧撑3/组12/次肱三头肌:仰卧撑3/组12/次哑铃颈后屈臂3/组12/次腹部：仰卧起坐3/组20/次仰卧举腿3/组20/次星期', 'language': 'No language found.'}, page_content='健身房健身计划(精选5篇)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n个人计划年度计划月度计划明年计划班主任计划班级计划其他计划商业计划'),
 Document(metadata={'source': 'http://www.plansum.cn/gerenjihua/73374.html', 'title': '健身房健身计划(精选5篇)', 'description': '胸部、肱三头肌、腹部胸部：杠铃卧推3/组12/次哑铃飞鸟3/组12/次俯卧撑3/组12/次肱三头肌:仰卧撑3/组12/次哑铃颈后屈臂3/组12/次腹部：仰卧起坐3/组20/次仰卧举腿3/组20/次星期', 'language': 'No language found.'}, page_content='计划总结网 > 工作计划 > 个人计划 >\n\n\n\n\n健身房健身计划\n\n时间：2024-08-14 15:53:13\n小龙\n个人计划'),
 Document(metadata={'source': 'http://www.plansum.cn/gerenjihua/73374.html', 'title': '健身房健身计划(精选5篇)', 'description': '胸部、肱三头肌、腹部胸部：杠铃卧推3/组12/次哑铃飞鸟3/组12/次俯卧撑3/组12/次肱三头肌:仰卧撑3/组12/次哑铃颈后屈臂3/组12/次腹部：仰卧起坐3/组20/次仰卧举腿3/组20/次星期', 'language': 'No language found.'}, page_content='胸部、肱三头肌、腹部胸部：杠铃卧推 3/组

In [19]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

# Step 3: Vector Store
Alternatively, we could use a cloud-based vector database.

In [4]:
vectorstore_in_memory = FAISS.from_documents(splits, embeddings)

# Step 4: Retrieval

In [9]:
query = "如何使用哑铃进行健身？"
top_k = 4
retrieved_docs = vectorstore_in_memory.similarity_search(query, k=top_k)

# Combine context
context = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [10]:
retrieved_docs

[Document(id='ce8483e1-b7a3-4c67-9fbc-595a8702029d', metadata={'source': 'http://www.plansum.cn/gerenjihua/73374.html', 'title': '健身房健身计划(精选5篇)', 'description': '胸部、肱三头肌、腹部胸部：杠铃卧推3/组12/次哑铃飞鸟3/组12/次俯卧撑3/组12/次肱三头肌:仰卧撑3/组12/次哑铃颈后屈臂3/组12/次腹部：仰卧起坐3/组20/次仰卧举腿3/组20/次星期', 'language': 'No language found.'}, page_content='健身房健身计划(精选5篇)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n个人计划年度计划月度计划明年计划班主任计划班级计划其他计划商业计划'),
 Document(id='b589fe43-f663-42e2-812f-7b053c241f49', metadata={'source': 'http://www.plansum.cn/gerenjihua/73374.html', 'title': '健身房健身计划(精选5篇)', 'description': '胸部、肱三头肌、腹部胸部：杠铃卧推3/组12/次哑铃飞鸟3/组12/次俯卧撑3/组12/次肱三头肌:仰卧撑3/组12/次哑铃颈后屈臂3/组12/次腹部：仰卧起坐3/组20/次仰卧举腿3/组20/次星期', 'language': 'No language found.'}, page_content='计划总结网 > 工作计划 > 个人计划 >\n\n\n\n\n健身房健身计划\n\n时间：2024-08-14 15:53:13\n小龙\n个人计划'),
 Document(id='6f3ced65-da4f-4f3e-9a27-722eb3244485', metadata={'source': 'http://www.plansum.cn/gerenjihua/73374.html', 'title': '健身房健身计划(精选5篇)', 'description': '胸部、肱三头肌、腹部胸部：杠铃卧推3/组12/次哑铃飞鸟3/组12/次俯卧撑3/组12/

In [7]:
context

'杠铃反弯举 12、10、10、12哑铃腕屈伸\n\n颈后推举： 8、6、8、8 前平举：12、10、10、12\n\n拉力侧平举15、20、15前臂内侧：杠铃腕弯举15、12、15哑铃单手腕弯举15、20、15下背：屈膝硬拉8、68 负重伸背12、10、10、12星期五肩、肱三头肌、腹肌肩： 颈后推举：\n\n拉力侧平举：15、15、15提肘拉：10、8、8、10肱三头肌：双杠臂屈伸10、8、10头后臂屈伸：10、8、10拉力下压臂屈伸：15、12、12俯立单臂屈伸：15、20、20、15腹：坐姿屈膝收腹'

# Step 5: Generate

In [11]:
# Get answer from DeepSeek
answer = call_deepseek(query, context)
print("\n DeepSeek Answer:\n", answer)


 DeepSeek Answer:
 使用哑铃进行健身是一种非常有效的方式，可以增强肌肉力量、改善体型并提升整体健康水平。以下是一些基本的哑铃健身方法和建议：

### 1. **选择合适的重量**
   - 初学者应从较轻的重量开始，逐渐增加重量以避免受伤。
   - 选择合适的重量，使得在完成一组动作（通常8-12次）时感到有些吃力，但仍能保持正确的姿势。

### 2. **基本哑铃动作**
   - **哑铃深蹲**：双手持哑铃放在身体两侧或肩部，进行深蹲动作，锻炼腿部肌肉。
   - **哑铃卧推**：躺在平板凳上，双手持哑铃向上推举，锻炼胸肌和手臂。
   - **哑铃划船**：弯腰，双手持哑铃向上拉，锻炼背部肌肉。
   - **哑铃肩推**：双手持哑铃，从肩部向上推举，锻炼肩部肌肉。
   - **哑铃弯举**：双手持哑铃，弯曲肘部将哑铃举向肩部，锻炼肱二头肌。

### 3. **注意事项**
   - **保持正确的姿势**：错误的姿势可能导致受伤，建议在开始前学习正确的动作或请教专业教练。
   - **控制动作速度**：避免快速甩动哑铃，应缓慢控制动作，确保肌肉充分参与。
   - **呼吸控制**：用力时呼气，放松时吸气，保持呼吸顺畅。
   - **热身和拉伸**：在训练前进行5-10分钟的热身，训练后进行拉伸，以减少受伤风险并促进恢复。

### 4. **训练计划**
   - 每周进行2-3次哑铃训练，每次训练选择4-6个动作，每个动作做3-4组，每组8-12次。
   - 根据个人目标（增肌、减脂或增强耐力）调整重量和次数。

### 5. **安全提示**
   - 训练时确保周围有足够的空间，避免碰撞。
   - 如果感到任何不适或疼痛，应立即停止训练并咨询医生或专业教练。

通过坚持科学的哑铃训练计划，你可以有效提升肌肉力量和体能水平。
